Allison Forte

Assignemnt 9.2: (11.1, 11.3, 11.4)

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

# 11.1
Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [56]:
import os
os.getcwd()
os.chdir('/Users/allison.forte/downloads/ThinkStats2-master/code')

In [57]:
import numpy as np
import pandas as pd
import thinkstats2
import thinkplot
import statsmodels.formula.api as smf
import nsfg
import first

live, firsts, others = first.MakeFrames()

In [58]:
live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on = 'caseid', rsuffix = '_r')
join.shape

(8884, 3331)

In [59]:
t = []
for name in join.columns:
    try:
        if join[name].var() < 1e-7:
            continue

        formula = 'prglngth ~ nbrnaliv>0 + ' + name
        model = smf.ols(formula, data = join)
        if model.nobs < len(join)/2:
             continue

        results = model.fit()
    except (ValueError, TypeError):
        continue
            
    t.append((results.rsquared,name))
        
t.sort(reverse = True)
for mse,name in t[:25]:
    print(name,mse)

prglngth 1.0
wksgest 0.8062442137869668
totalwgt_lb 0.12445743148120225
birthwgt_lb 0.11977307804917148
lbw1 0.10372891564177522
mosgest 0.09566165395747539
prglngth_i 0.022106720422427895
canhaver 0.00624990269934711
datcon01_i 0.005870896928044322
con1mar1_i 0.00559992223864747
nbrnaliv 0.004577565785532256
mar1con1_i 0.0032028472281693254
anynurse 0.0024520248837121006
bfeedwks 0.0024481839250949378
pregend1 0.0023063144297525984
rmarout11_i 0.0022934337962573492
marout11_i 0.0022934337962573492
marcon11_i 0.0022934337962573492
cmlastlb_r 0.002092483922756516
cmlastlb 0.002092483922756516
datend02_i 0.0020627691771459844
datcon02_i 0.0020627691771459844
evuseint 0.0020394950463963335
agecon02_i 0.002038765443930335
fmarcon5_i 0.0020211427836561713


In [60]:
import re

def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=25):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

MiningReport(t)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8062442137869668 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb 0.12445743148120225
birthwgt_lb 0.11977307804917148 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.10372891564177522 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09566165395747539 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.022106720422427895 PRGLNGTH IMPUTATION FLAG
canhaver 0.00624990269934711 DF-1 PHYSICALLY DIFFICULT FOR R TO HAVE A BABY
datcon01_i 0.005870896928044322 DATCON01 IMPUTATION FLAG
con1mar1_i 0.00559992223864747 CON1MAR1 IMPUTATION FLAG
nbrnaliv 0.004577565785532256 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
mar1con1_i 0.0032028472281693254 MAR1CON1 IMPUTATION FLAG
anynurse 0.0024520248837121006 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.0024481839250949378 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.0023063144297525984 BC-1 HOW PREGNANCY ENDED - 1ST ME

In [61]:
# Based on the above code, there are many variables that correlate with length of pregnancy.
# Filtering out variables that are not known before birth or not generally available, a few stand out.
# Variable 'canhaver' indicating whether it is physically difficult for the individual to have a child is very relevant but likely unknown. 
# Presumably we know our coworkers race and can use that in our estimate.
# We should also note whether this is a first child or not. If this is a first child we can use birthord == 1, otherwise birthord != 1. 
# We can also include the nbrnaliv variable set >0 as we are assuming we there will be at least 1 baby born.

# We can use the formula below with the birthord, race, and nbrnaliv variables adjusted for the coworker's situation.
model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>0', data=join)
results = model.fit()
results.summary()

# Comparing the results from the above model to a model without the race and nbrnaliv variables, we can double our chances of picking the right date. 
# We can increase the chance from 1% to 2% by adding in the 2 additional variables to the model. 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     6.813
Date:                Wed, 11 May 2022   Prob (F-statistic):           0.000139
Time:                        15:33:57   Log-Likelihood:                -18288.
No. Observations:                8884   AIC:                         3.658e+04
Df Residuals:                    8880   BIC:                         3.661e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.1861      0.948     40.271      0.000      36.327      40.045
birthord == 1[T.True]     0.1185      0.040      2.941      0.003       0.040       0.198
race == 2[T.True]         0.1370      0.042      3.249      0.001       0.054       0.220
nbrnaliv > 0[T.True]      0.5463      0.949      0.576      0.565      -1.313       2.406
==============================================================================
Omnibus:                     1619.596   Durbin-Watson:                   1.628
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6254.476
Skew:                          -0.871   Prob(JB):                         0.00
Kurtosis:                       6.724   Cond. No.                         110.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 11.3
If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called poisson. It works the same way as ols and logit. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called numbabes.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [62]:
# First we create the model and review the results

model = smf.poisson('numbabes ~ age_r + C(race) + totincr + educat', data=join)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.687055
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Wed, 11 May 2022   Pseudo R-squ.:                 0.03109
Time:                        15:36:03   Log-Likelihood:                -14988.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.106e-205
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.0842      0.045     23.995      0.000       0.996       1.173
C(race)[T.2]    -0.1398      0.015     -9.464      0.000      -0.169      -0.111
C(race)[T.3]    -0.0914      0.025     -3.717      0.000      -0.140      -0.043
age_r            0.0208      0.001     20.474      0.000       0.019       0.023
totincr         -0.0179      0.002     -9.442      0.000      -0.022      -0.014
educat          -0.0443      0.003    -15.139      0.000      -0.050      -0.039
================================================================================
"""

In [63]:
# Next we create a data frame with the relevant columns and data for our individual
# When we use our model on the new data frame, we get our results

new = pd.DataFrame([[35, 1, 14, 16]], columns=('age_r', 'race', 'totincr', 'educat'))
results.predict(new)

0    2.342182
dtype: float64

We can conclude that the woman has likely born 2.34 children or, more meaningfully, she has likely born 2 children when we round the results. 

# 11.4
If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called mnlogit. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called rmarital.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [64]:
# First we create the model and review the results

model = smf.mnlogit('rmarital ~ age_r + C(race) + totincr + educat', data=join)
results2 = model.fit()
results2.summary()

Optimization terminated successfully.
         Current function value: 1.087603
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Wed, 11 May 2022   Pseudo R-squ.:                  0.1655
Time:                        15:37:23   Log-Likelihood:                -9662.3
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.4532      0.279     15.977      0.000       3.907       5.000
C(race)[T.2]    -0.9219      0.089    -10.409      0.000      -1.095      -0.748
C(race)[T.3]    -0.6334      0.136     -4.674      0.000      -0.899      -0.368
age_r           -0.0570      0.006     -9.754      0.000      -0.068      -0.046
totincr         -0.1302      0.012    -11.298      0.000      -0.153      -0.108
educat          -0.2051      0.019    -11.017      0.000      -0.242      -0.169
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.5432      0.916     -4.960      0.000      -6.338      -2.748
C(race)[T.2]    -0.4405      0.236     -1.865      0.062      -0.904       0.023
C(race)[T.3]     0.0329      0.335      0.098      0.922      -0.623       0.689
age_r            0.1301      0.019      6.833      0.000       0.093       0.167
totincr         -0.3251      0.032    -10.182      0.000      -0.388      -0.263
educat          -0.1010      0.048     -2.100      0.036      -0.195      -0.007
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.6485      0.302     -8.783      0.000      -3.239      -2.057
C(race)[T.2]    -0.3188      0.093     -3.417      0.001      -0.502      -0.136
C(race)[T.3]    -0.7664      0.171     -4.485      0.000      -1.101      -0.431
age_r            0.0641      0.007      9.817      0.000       0.051       0.077
totincr         -0.2270      0.012    -19.588      0.000      -0.250      -0.204
educat           0.0673      0.017      4.036      0.000       0.035       0.100
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8231      0.345      2.385      0.017       0.147       1.500
C(race)[T.2]    -1.0491      0.104    -10.125      0.000      -1.252      -0.846
C(race)[T.3]    -0.5644      0.156     -3.624      0.000      -0.870      -0.259
age_r            0.0063      0.007      0.872      0.383      -0.008       0.020
totincr         -0.2923      0.015    -20.149      0.000      -0.321      -0.264
educat          -0.0136      0.021     -0.639      0.523      -0.056       0.028
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.2370      0.275     22.654      0.000       5.697       6.777
C

In [65]:
# Next we create a data frame with the relevant columns and data for our individual
# When we use our model on the new data frame, we get our results

new2 = pd.DataFrame([[25, 2, 11, 12]], columns=('age_r', 'race', 'totincr', 'educat'))
results2.predict(new2)

,0,1,2,3,4,5
0,0.748384,0.125474,0.001103,0.035295,0.023813,0.065931


Based on our results we can conclude that this woman has a 74.8% chance of being being currently married to a person of the opposite sex (column 0). She has a 12.5% chance of currently living with an opposit sex partner while not married (column 1). There is only a .1% chance she is widowed (column 2) and a 3.5% chance that she is divorced or had a marriage annulled (column 3). Finally there is a 2.4% chance she is separated for reasons of marital discord (column 4) and a 6.6% chance she has never been married (column 5).